In [22]:
# Delta Version - Image Generator

import numpy as np
import matplotlib.pyplot as plt
from scipy.special import beta
import os
import cv2

# Set the seed for reproducibility
np.random.seed(123)

def thrown_ball(n=3):
    x_position = round(np.random.rand(), n)
    y_position = round(np.random.rand(), n)
    return x_position, y_position

def update_counts(white_position, red_position, left, right, bottom, top, overlap):
    x_white, y_white = white_position
    x_red, y_red = red_position
    
    # Overlap case
    if (x_white == x_red) and (y_white == y_red):
        overlap += 1
    
    # X case
    if x_red < x_white:
        left += 1
    elif x_red > x_white:
        right += 1
    
    # Y case
    if y_red < y_white:
        bottom += 1
    elif y_red > y_white:
        top += 1
    
    return left, right, bottom, top, overlap

# Function to calculate the probability density function (PDF) of the beta distribution
def calculate_PDF(A, B, p):
    numerator = (p ** A) * ((1 - p) ** B)
    denominator = beta(A + 1, B + 1)
    return numerator/denominator

def generate_heatmap_plot(white_ball, red_ball, left_count, right_count, bottom_count, top_count, overlap_count):
    # Parameters
    p_values = np.linspace(0, 1, 1000)

    # Calculate PDFs
    pdf_x = calculate_PDF(left_count, right_count, p_values)
    pdf_y = calculate_PDF(bottom_count, top_count, p_values)

    # Create a grid of p values
    p_mesh, _ = np.meshgrid(p_values, p_values)

    # Create a grid for summing values at each point
    grid_prod = np.outer(pdf_y, pdf_x)

    # White position
    x_white, y_white = white_ball

    # Plotting the heatmap
    fig, ax = plt.subplots()
    im = ax.imshow(grid_prod, extent=(0, 1, 0, 1), origin='lower', aspect='auto', cmap='viridis')
    cbar = plt.colorbar(im, ax=ax, label='Probability Density')
    ax.set_xlabel('p')
    ax.set_ylabel('p')
    ax.set_title('Probability Density Function Heatmap')

    # Adding a point at the white position
    ax.scatter(x_white, y_white, color='red', marker='.', label='White Ball')

    ax.legend()

    return fig

# Initialize counts
left_count, right_count, bottom_count, top_count, overlap_count = 0, 0, 0, 0, 0

# Number of balls
num_balls = 100

# Set the desired interval to print messages
print_interval = 50

# Simulate throwing a white ball
white_ball = thrown_ball()

# Create a 'figures' directory if it doesn't exist
figures_directory = 'figures'
os.makedirs(figures_directory, exist_ok=True)

# Simulate throwing balls and update counts
for i in range(num_balls):
    red_ball = thrown_ball()

    left_count, right_count, bottom_count, top_count, overlap_count = update_counts(
        white_ball, red_ball, left_count, right_count, bottom_count, top_count, overlap_count
    )
    
    plot_fig = generate_heatmap_plot(white_ball, red_ball, left_count, right_count, bottom_count, top_count, overlap_count)

    # Save the figure inside the 'figures' directory
    figure_path = os.path.join(figures_directory, f'figure_{i}.png')
    plt.savefig(figure_path)
    plt.close()
    
    # Print a message every print_interval iterations
    if i % print_interval == 0:
        print(f"Processed {i} out of {num_balls} iterations")

print("Simulation and figure generation completed.")

Processed 0 out of 100 iterations
Processed 50 out of 100 iterations
Simulation and figure generation completed.


In [23]:
import cv2
import os

# Set the path to the directory containing your figure images
images_directory = 'figures'

# Set the output video file name and path
output_video_path = 'bayesian_billiards_simulation.mp4'

# Get the list of image files in the directory and sort them numerically
image_files = sorted([f for f in os.listdir(images_directory) if f.endswith('.png')], key=lambda x: int(x.split('_')[1].split('.')[0]))

# Get the dimensions of the first image
first_image = cv2.imread(os.path.join(images_directory, image_files[0]))
height, width, layers = first_image.shape

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can choose other codecs like 'XVID' or 'MJPG'
video = cv2.VideoWriter(output_video_path, fourcc, 30, (width, height))

# Iterate through the sorted images and write them to the video
for image_file in image_files:
    image_path = os.path.join(images_directory, image_file)
    img = cv2.imread(image_path)
    video.write(img)

# Release the VideoWriter object
video.release()

print(f"Video created successfully at: {output_video_path}")

Video created successfully at: bayesian_billiards_simulation.mp4
